In [1]:
# 시스템 모듈
import os
import yaml
from dotenv import load_dotenv

# 랭체인 모듈
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

# 환경변수 로드
load_dotenv()

# 매뉴얼 파일 및 캐싱 데이터 주소
manual_data_dir = './docs'
cache_dir = LocalFileStore(f"./.cache/embeddings/data")


# 매뉴얼 파일 열기 및 읽기
def load_and_parse_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    return content 

# 매뉴얼 파일 내의 YAML frontmatter 추출(Metadata)
def extract_front_matter(content):
    if content.startswith('---'):
        end = content.find('---', 3)
        if end != -1:
            front_matter = content[3:end].strip()
            try:
                metadata = yaml.safe_load(front_matter)
                return metadata
            except yaml.YAMLError as e:
                print(f"Error parsing YAML: {e}") 
    return {} 

# 스플리터 설정
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=800,
    chunk_overlap=200,
)

# 매뉴얼 파일 로드 및 처리
def process_markdown_file(file_path):
    content = load_and_parse_file(file_path)
    metadata = extract_front_matter(content)
    
    loader = UnstructuredFileLoader(file_path)
    documents = loader.load()
    
    if documents:
        document = documents[0]
        document.metadata.update(metadata)
        # 텍스트를 분할하고 메타데이터 포함
        split_documents = splitter.create_documents([document.page_content], [document.metadata])
        return split_documents
    return []

# 모든 마크다운 파일을 처리하여 docs 리스트에 저장
docs = []

for root, _, files in os.walk(manual_data_dir):
    for file in files:
        if file.endswith(".md"):
            file_path = os.path.join(root, file)
            documents = process_markdown_file(file_path)
            if documents:
                docs.extend(documents)  # 분할된 각 문서를 개별적으로 추가


print(f"Total documents: {len(docs)}")

# 임베딩 및 백터 저장소 생성
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
vectorstore = FAISS.from_documents(docs, cached_embeddings)
DB_INDEX = "MANUAL_DB"
vectorstore.save_local(DB_INDEX)

Processing completed. Total documents: 106
